# Домашняя работа №5. Метод опорных векторов

Цель работы: Использовать метод опорных векторов для решения задачи регрессии.

### Работа с набором данных для задачи регрессии (целевая переменная - непрерывное значение)

## Задание №1

#### Предварительная подготовка данных

Подготовить данные (если не выполнялось ранее, иначе использовать подготовленные данные):

* Заполнить пропуски в данных подходящими значениями (например, 0 для числовых значений, other для категориальных значений).
* Проверить датасет на дубликаты. Удалить при наличии.
* Закодировать категориальные значения. Например, создать новые булевые признаки (OneHotEncoder), или назначить число для каждого значения (LabelEncoder).
* Удалить столбцы с уникальными значениями, при наличии.
* К числовым признакам применить масштабирование со средним нулевым значением.
* Разделить на тренировочную и обучающую выборки.

## Задание №2

#### Обучение модели
* Обучить модель с помощь класса SVR в Sklearn с параметрами по умолчанию.
* Вычислить метрики r2, mse, mae на тренировочной и тестовой выборках.
* Сделать вывод о качестве обученной модели.

## Задание №3

#### Настройка гиперпараметров модели
* С помощью инструмента Optuna подобрать оптимальные гиперпараметры (ядро: Полиномиальное, Радиальное; С: от 1 до 100, гамма: 0.01 до 100)
* С помощью найденных лучших параметров обучить модель.
* Вычислить метрики полученной модели (r2, mse, mae) на тренировочной и тестовой выборках.

# Решение домашней работы №5

In [3]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.4/364.4 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 5.9 MB/s eta 0:00:00


In [4]:
!pip install --upgrade scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 44.2 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.5.2
    Uninstalling scikit-learn-1.5.2:
      Successfully uninstalled scikit-learn-1.5.2


In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.svm import SVR
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import optuna

In [7]:
df = pd.read_csv('/content/train.csv')

##Задание №1

Обработка дубликатов и пропусков

In [8]:
print(df.isnull().sum())

datetime      0
season        0
holiday       0
workingday    0
weather       0
temp          0
atemp         0
humidity      0
windspeed     0
casual        0
registered    0
count         0
dtype: int64


In [9]:
if len(df) != len(df.drop_duplicates()):
    df = df.drop_duplicates()

Преобразуем категориальные признаки в числовой формат

In [11]:
categorical_features = ['season', 'holiday', 'workingday', 'weather']
ohe = OneHotEncoder(sparse=False, handle_unknown='ignore')
encoded_data = ohe.fit_transform(df[categorical_features])

encoded_df = pd.DataFrame(encoded_data, columns=ohe.get_feature_names_out(categorical_features))

df = df.drop(columns=categorical_features).join(encoded_df)

TypeError: OneHotEncoder.__init__() got an unexpected keyword argument 'sparse'